# 05. PyTorch Going Modular Exercises

## 0. Resources

These exercises/solutions are based on section 05. PyTorch Going Modular of the Learn PyTorch for Deep Learning course by Zero to Mastery: https://www.learnpytorch.io/05_pytorch_going_modular/

## 1. Turn the code to get the data into a Python script

When you run the script using python `get_data.py` it should check if the data already exists and skip downloading if it does.
If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the data directory.

In [1]:
%%writefile src/get_data.py
"""
Contains function to download data
"""

import os
import requests
import zipfile
from pathlib import Path


def get_data(
    data_dir_str: str = "data/",
    image_path_str: str = "pizza_steak_sushi",
    data_url_str: str = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
    file_name_str: str = "pizza_steak_sushi.zip"
) -> None:
    """Downloads data from GitHub.

    Args:
        data_dir_str (str, optional): Path do data directory.
            Defaults to "../data/".
        image_path_str (str, optional): Name of the folder where data will
            be stored. Defaults to "pizza_steak_sushi".
        data_url_str (_type_, optional): Link to site from where data will
            be downloaded. Defaults to "https://github.com/mrdbourke/ \
            pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip".
        file_name_str (str, optional): Name of the downloaded file.
            Defaults to "pizza_steak_sushi.zip".
    """
    
    # Setup path to data folder
    data_dir = Path(data_dir_str)
    image_path = data_dir / image_path_str
    
    # Check if data folder exists
    if image_path.exists():
        print(f"{image_path} exists...")
    else:
        print(f"{image_path} does not exists, creating...")
        image_path.mkdir(parents=True, exist_ok=True)
    
    # Check if data is already downloaded
    if len(list(image_path.glob("*/*/*"))) == 0:
        
        # Download data
        with open(data_dir / file_name_str, "wb") as f:
            print(f"Downloading {file_name_str}...")
            request = requests.get(data_url_str)
            f.write(request.content)
        
        # Unzip data
        with zipfile.ZipFile(data_dir / file_name_str, "r") as z:
            print(f"Extracting {file_name_str}...")
            z.extractall(image_path)
            
        # Remove zip file
        print(f"Deleting {file_name_str}...")
        os.remove(data_dir / file_name_str)
    else:
        print(f"Data in {image_path} already exits, skipping downloading and unzipping...")
        
    print("Finished getting data...")


if __name__ == "__main__":
    get_data()

Overwriting src/get_data.py


In [2]:
# Example running of get_data.py
!python src/get_data.py

data/pizza_steak_sushi exists...
Data in data/pizza_steak_sushi already exits, skipping downloading and unzipping...
Finished getting data...


## 2. Create Datasets and DataLoaders

In [3]:
%%writefile src/setup_data.py
"""
Contains functino to setup datasets and dataloaders
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = 0
BATCH_SIZE = 32

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int = BATCH_SIZE,
    num_workers: int = NUM_WORKERS,
):
    """Creates training and testing DataLoaders.

    Args:
        train_dir (str): Path to training directory.
        test_dir (str): Path to testing directory.
        transform (transforms.Compose): torchvision.transforms to perform
            on training and testing data. Defaults to BATCH_SIZE.
        batch_size (int, optional): Number of samples per batch in
            each of the DataLoaders.
        num_workers (int, optional): Integer for number of workers
            per DataLoader.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names) 
        where class_names is a list of the target classes.
    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(
        root=train_dir,
        transform=transform,
        target_transform=None,
    )
    test_data = datasets.ImageFolder(
        root=test_dir,
        transform=transform,
        target_transform=None,
    )
    
    # Get class names as a list
    class_names = train_data.classes
    
    # Turn train and test Datasets into DataLoaders
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=True,
    )
    
    return train_dataloader, test_dataloader, class_names

Overwriting src/setup_data.py


In [4]:
from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor(),
])

from src import setup_data
from pathlib import Path

train_dir = Path("data") / "pizza_steak_sushi" / "train"
test_dir = Path("data") / "pizza_steak_sushi" / "test"

train_dataloader, test_dataloader, class_names = setup_data.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform
)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x11836e270>,
 <torch.utils.data.dataloader.DataLoader at 0x11834b250>)

In [5]:
len(train_dataloader), len(test_dataloader)

(8, 3)

## 3. Create a model (TinyVGG)

In [6]:
%%writefile src/model_builder.py
"""
Contains model code to instantiate TinyVGG model
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
    """Creates TinyVGG architecture.

        Args:
        input_shape: An integer indicating number
            of input channels.
        hidden_units: An integer indicating number
            of hidden units between layers.
        output_shape: An integer indicating number
            of output units.
    """
    
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=input_shape,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2,
                stride=2,
            ),
        )
        
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                stride=1,
                padding=0,
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2,
                stride=2,
            ),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(
                in_features=hidden_units*13*13,
                out_features=output_shape,
            ),
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

Overwriting src/model_builder.py


In [7]:
import torch
from src import model_builder

device = "cuda" if torch.cuda.is_available() else (
    "mps" if torch.mps.is_available() else "cpu"
)

# Instantiate a mode from the model_builder.py script
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=16,
    output_shape=len(class_names)
).to(device)

model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2704, out_features=3, bias=True)
  )
)

In [8]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model.eval()
with torch.inference_mode():
    pred = model(img_single.to(device))

# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(
    f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

/Users/mchojna/Documents/GitHub/pytorch-course/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[0.0258, 0.0559, 0.0569]], device='mps:0')

Output prediction probabilities:
tensor([[0.3266, 0.3366, 0.3369]], device='mps:0')

Output prediction label:
tensor([2], device='mps:0')

Actual label:
2


## 4. Turn training and testing functions into scripts

In [9]:
%%writefile src/engine.py
"""
Contains functions for training and testing model.
"""

from typing import Dict, List, Tuple
from tqdm.auto import tqdm
import torch

def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str
) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained.
        dataloader (torch.utils.data.DataLoader): A DataLoader instance
            for the model to be trained on.
        loss_fn (torch.nn.Module): A PyTorch loss function
        to minimize.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to
            help minimize the loss function.
        device (str): A target device to compute 
            on (e.g. "cuda" or "cpu").


    Returns:
        Tuple[float, float]: A tuple of training loss and
            training accuracy metrics.
    """
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0., 0.
    
    # Loop through data loader data batches
    for X, y in dataloader:
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(
    model: torch.nn.Module, 
    dataloader: torch.utils.data.DataLoader, 
    loss_fn: torch.nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Args:
        model (torch.nn.Module): A PyTorch model to be tested.
        dataloader (torch.utils.data.DataLoader): A DataLoader instance 
            for the model to be tested on.
        loss_fn (torch.nn.Module): A PyTorch loss function to calculate
            loss on the test data.
        device (torch.device): A target device to compute on
            (e.g. "cuda" or "cpu").


    Returns:
        Tuple[float, float]: A tuple of testing loss and
            testing accuracy metrics.
    """
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for X, y in dataloader:
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(
    model: torch.nn.Module, 
    train_dataloader: torch.utils.data.DataLoader, 
    test_dataloader: torch.utils.data.DataLoader, 
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device
) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained
            and tested.
        train_dataloader (torch.utils.data.DataLoader): A DataLoader
            instance for the model to be trained on.
        test_dataloader (torch.utils.data.DataLoader): A DataLoader
            instance for the model to be tested on.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to
            help minimize the loss function.
        loss_fn (torch.nn.Module): A PyTorch loss function to
            calculate loss on both datasets.
        epochs (int): An integer indicating how many epochs
            to train for.
        device (torch.device): A target device to compute
            on (e.g. "cuda" or "cpu").

    Returns:
        Dict[str, List[float]]: A dictionary of training and
        testing loss as well as training and
        testing accuracy metrics. Each metric has a value in
        a list for each epoch.
    """
    # Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                            dataloader=train_dataloader,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)
        
        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Writing src/engine.py


In [12]:
from src import engine

engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=torch.optim.SGD(params=model.parameters(), lr=0.001),
    loss_fn=torch.nn.CrossEntropyLoss(),
    epochs=3,
    device=device,
)

  0%|          | 0/3 [00:00<?, ?it/s]

/Users/mchojna/Documents/GitHub/pytorch-course/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch: 1 | train_loss: 1.0991 | train_acc: 0.3633 | test_loss: 1.0967 | test_acc: 0.4422
Epoch: 2 | train_loss: 1.0979 | train_acc: 0.3750 | test_loss: 1.0967 | test_acc: 0.4119
Epoch: 3 | train_loss: 1.0967 | train_acc: 0.3867 | test_loss: 1.0972 | test_acc: 0.4347


{'train_loss': [1.099069595336914, 1.097921147942543, 1.0966666787862778],
 'train_acc': [0.36328125, 0.375, 0.38671875],
 'test_loss': [1.096712867418925, 1.0967222452163696, 1.097179651260376],
 'test_acc': [0.44223484848484845, 0.4119318181818182, 0.4346590909090909]}

## 5. Turn saving functions into scripts

In [14]:
%%writefile src/utils.py
"""
File containint utility functions for model training
"""

from pathlib import Path
import torch

def save_model(
    model: torch.nn.Module,
    target_dir: str,
    model_name: str
):
    """Saves a PyTorch model to a target directory.


    Args:
        model (torch.nn.Module): A target PyTorch model to save.
        target_dir (str): A directory for saving the model to.
        model_name (str): A filename for the saved model. Should include
            either ".pth" or ".pt" as the file extension.
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
                f=model_save_path)

Writing src/utils.py


## 6. Train evaluate and save model

In [30]:
%%writefile src/train.py
"""
Trains a PyTorch image classification model using device-agnostic code 
"""

import os
import torch
from torchvision import transforms
import setup_data, engine, model_builder, utils
from timeit import default_timer as timer

# Setup hyperparamaters
NUM_EPOCHS = 20
BATCH_SIZE = 16
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else (
    "mps" if torch.mps.is_available else "cpu"
)

if __name__ == "__main__":

    # Create transforms
    data_transform = transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.ToTensor(),
    ])

    # Create DataLoaders and get class_names
    test_dataloader, train_dataloader, class_names = setup_data.create_dataloaders(
        train_dir=train_dir,
        test_dir=test_dir,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    # Create model
    model = model_builder.TinyVGG(
        input_shape=3,
        hidden_units=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    # Setup loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=LEARNING_RATE,
    )

    # Start timer
    start_time = timer()

    # Start trainig with help from engine.py
    engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=NUM_EPOCHS,
        device=device,
    )

    # End timer
    end_time = timer()
    print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

    # Save the model to file
    utils.save_model(
        model=model,
        target_dir="models",
        model_name="05_pytorch_going_modular_script_mode.pth"
    )

Overwriting src/train.py


In [31]:
!python src/train.py

  0%|                                                    | 0/20 [00:00<?, ?it/s]/Users/mchojna/Documents/GitHub/pytorch-course/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch: 1 | train_loss: 1.0927 | train_acc: 0.4216 | test_loss: 1.0968 | test_acc: 0.3625
100%|███████████████████████████████████████████| 20/20 [00:18<00:00,  1.07it/s]
[INFO] Total training time: 18.668 seconds
[INFO] Saving model to: models/05_pytorch_going_modular_script_mode.pth


## 7. Use `argparse` module to be able to send the `train.py` custom hyperparameter values for training procedures

Add an argument flag for using a different:
- Training/testing directory
- Learning rate
- Batch size
- Number of epochs to train for
- Number of hidden units in the TinyVGG model
    - Keep the default values for each of the above arguments as what they already are

For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`

In [73]:
test_dir, train_dir

(PosixPath('data/pizza_steak_sushi/test'),
 PosixPath('data/pizza_steak_sushi/train'))

In [88]:
%%writefile src/test_arg.py
"""
Trains a PyTorch image classification model using device-agnostic code and custom arguments
"""
import argparse

parser = argparse.ArgumentParser(
    prog="ProgramName",
    description="What the program does",
    epilog="Text at the bottom of help",
)

parser.add_argument("-r", "--train_dir", default="data/pizza_steak_sushi/train")
parser.add_argument("-s", "--test_dir", default="data/pizza_steak_sushi/test")
parser.add_argument("-l", "--learning_rate", default="0.001")
parser.add_argument("-b", "--batch_size", default="32")
parser.add_argument("-e", "--num_epochs", default="8")
parser.add_argument("-u", "--hidden_units", default="8")
parser.add_argument("-n", "--model_name", default="model.pth")

args = parser.parse_args()

TRAIN_DIR = args.train_dir
TEST_DIR = args.test_dir
LEARNING_RATE = float(args.learning_rate)
BATCH_SIZE = int(args.batch_size)
NUM_EPOCHS = int(args.num_epochs)
HIDDEN_UNITS = int(args.hidden_units)
MODEL_NAME = args.model_name

print(f"Paths:\n\tTrain directory: {TRAIN_DIR}\n\tTest directoryt: {TEST_DIR}\n\tModel directory: models/{MODEL_NAME}\n")
print(f"Hyperparameters:\n\tLearning rage: {LEARNING_RATE}\n\tBatch size: {BATCH_SIZE}\n\tNumber of epochs: {NUM_EPOCHS}\n\tHidden units: {HIDDEN_UNITS}\n")

import os
import torch
from torchvision import transforms
import setup_data, engine, model_builder, utils
from timeit import default_timer as timer

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else (
    "mps" if torch.mps.is_available else "cpu"
)

if __name__ == "__main__":

    # Create transforms
    data_transform = transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.ToTensor(),
    ])

    # Create DataLoaders and get class_names
    test_dataloader, train_dataloader, class_names = setup_data.create_dataloaders(
        train_dir=TRAIN_DIR,
        test_dir=TEST_DIR,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )

    # Create model
    model = model_builder.TinyVGG(
        input_shape=3,
        hidden_units=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    # Setup loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=LEARNING_RATE,
    )

    # Start timer
    start_time = timer()

    # Start trainig with help from engine.py
    engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=NUM_EPOCHS,
        device=device,
    )

    # End timer
    end_time = timer()
    print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

    # Save the model to file
    utils.save_model(
        model=model,
        target_dir="models",
        model_name=MODEL_NAME
    )

Overwriting src/test_arg.py


In [89]:
!python src/test_arg.py -n model_2.pth

Paths:
	Train directory: data/pizza_steak_sushi/train
	Test directoryt: data/pizza_steak_sushi/test
	Model directory: models/model_2.pth

Hyperparameters:
	Learning rage: 0.001
	Batch size: 32
	Number of epochs: 8
	Hidden units: 8

  0%|                                                     | 0/8 [00:00<?, ?it/s]/Users/mchojna/Documents/GitHub/pytorch-course/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch: 1 | train_loss: 1.0972 | train_acc: 0.3608 | test_loss: 1.1002 | test_acc: 0.2930
100%|█████████████████████████████████████████████| 8/8 [00:07<00:00,  1.03it/s]
[INFO] Total training time: 7.806 seconds
[INFO] Saving model to: models/model_2.pth


## 8. Create a Python script to predict on a target image given a file path with a saved model

- For example, you should be able to run the command python `predict.py` `some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
- You may also have to write code to load in a trained model.